
# Amazon Rekoginiton Coustom Labelによる推論検証

In [ ]:
import boto3
import json
import os
import time
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from time import sleep


### AWS SDK Client

In [ ]:
#AWS SDK Client
REKOGNITION_CLIENT_NAME ='rekognition'
S3_CLIENT_NAME ='s3'

### Rekoginiton Amazon Resours Name

In [ ]:
#Amazon Resours Name
REKOGNITION_ARN ='arn:aws:rekognition:us-east-1:[You are rekognition_arn]'
PROJECT_ARN ='arn:aws:rekognition:us-east-1:[You are rekognition_project_arn]'
VERSION_NAME='[You are rekognition_model_version_name]'

### AWS Session Configure

In [ ]:
#AWS Configure
AWS_ACCESS_KEY_ID='[You are Access Key Id]'
AWS_SECRET_ACCESS_KEY='[You are Secret Access key Id]'
AWS_REGIN ='us-east-1' #東京リージョンではまだ開放されていません

### Boto3クライアントの作成

In [ ]:
rekognition_client = boto3.client(REKOGNITION_CLIENT_NAME,
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        region_name=AWS_REGIN
)
print(rekognition_client)

### 推論モデルの状態を確認

In [ ]:
#Recognitionプロジェクトの現在の状態
response =rekognition_client.describe_project_versions(
    ProjectArn=PROJECT_ARN,
    VersionNames=[VERSION_NAME,]
)
for pro_state in response["ProjectVersionDescriptions"]:
        if pro_state["ProjectVersionArn"] == REKOGNITION_ARN :
            print(pro_state["Status"])

### 推論モデルの起動
- Status:STARTING => 起動中 （推論実行不能）
- Status:RUNNIG => 起動（推論実行可能）
- 推論モデルを動かしている時間に対して課金 （＄4/hour) 
- 推論用インスタンスの実行までは5分から10分ほど待つ必要があります｡
### 必ず最後は停止させよう！！！

In [ ]:

is_running=False
is_stop = False
is_start_waite=False
timeout =100
timeout_cnt =0

while timeout> timeout_cnt :
    
    #Recognitionプロジェクトの状態を取得
    response =rekognition_client.describe_project_versions(
        ProjectArn=PROJECT_ARN,
        VersionNames=[VERSION_NAME,]
    )
    
    #RecognitionプロジェクトのステータスJSONを読み取り
    for pro_state in response["ProjectVersionDescriptions"]:
        
        if pro_state["ProjectVersionArn"] == REKOGNITION_ARN :
            
            #停止済み ＝＞ 起動開始
            if pro_state["Status"] == "STOPPED":
                print ("Is STOPPED now…")
                response = rekognition_client.start_project_version(
                    ProjectVersionArn=REKOGNITION_ARN,
                    MinInferenceUnits=1
                )

            #起動中のなので終わり
            if pro_state["Status"] == "RUNNING":
                print ("Is RUNNING now…")
                is_running = True

            #起動開始中のため待機
            if pro_state["Status"] == "STARTING":
                if not is_start_waite:
                    print ("Is STARTING wait…")
                    is_start_waite = True
                
            #終了中のため待機
            if pro_state["Status"] == "STOPPING":
                print ("Is STOPPING wait…")

            break
    
    #起動が確認できたら脱出
    if is_running :
        print("RUNNING:OK")
        break
    
    #5秒待機
    print("…" ,end="")
    sleep (5)
    timeout_cnt +=1

if not is_running:
    print("TIME OUT")

In [ ]:

if is_running :
    start_s3 = time.time()
        
    #推論用のS3を指定
    S3_BUCKET ="You are Bucket"
    S3_DIR_PATH ="You are Path"
    
    #推論処理を行いたいファイルパスを入れる(\ は /  に置換)
    DETECT_UPLOAD_FILE_PATH ="Detect Img file Path"
    DETECT_UPLOAD_FILE_PATH = DETECT_UPLOAD_FILE_PATH.replace(os.path.sep,'/')
    #ファイル名のみを抜出
    UPLOAD_FILE_NAME = os.path.basename(DETECT_UPLOAD_FILE_PATH)
    print(UPLOAD_FILE_NAME)
    
    #推論画像の読み込み
    im = Image.open(DETECT_UPLOAD_FILE_PATH)
    #推論画像をarrayに変換
    im_list = np.asarray(im)
    plt.imshow(im_list)
    #推論画像表示
    plt.show()
    
    #S3クライアント作成
    s3_client = boto3.client(S3_CLIENT_NAME,
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        region_name=AWS_REGIN,
    )
    
    #S3にアップロード
    response = s3_client.upload_file(DETECT_UPLOAD_FILE_PATH, S3_BUCKET, os.path.join(S3_DIR_PATH, UPLOAD_FILE_NAME))
    print(response)
    print(os.path.join(S3_DIR_PATH, UPLOAD_FILE_NAME))
    print()
    
    s3Upload_time = time.time() - start_s3
    print ("S3 Upload Time:{0}".format(s3Upload_time) + "[sec]")
    print()
    
    
    start_rekogonition = time.time()
    
    #Recgnittionで推論実行
    response = rekognition_client.detect_custom_labels(
        ProjectVersionArn=REKOGNITION_ARN,
        Image={
            'S3Object': {
                'Bucket': S3_BUCKET,
                'Name':  os.path.join(S3_DIR_PATH, UPLOAD_FILE_NAME),
            }
        },
    )
    
    #結果
    print(response)
    print()
    
    for res in response["CustomLabels"]:
        print("ラベル: " + res["Name"])
        print("信頼性: " + str(res["Confidence"]))
    
    rekogonition_detect_time = time.time() - start_rekogonition
    print()
    print ("Rekogonition Detect Time:{0}".format(rekogonition_detect_time) + "[sec]")
    


### 推論モデルの停止
- 推論モデルのステータスが「STARTING」の間は停止はかけられない
- ステータスが「RUNNING」になってから停止させる

In [ ]:
#無限ループでRUNNING OR STOP になるまで待機

is_running=False
is_stop = False
timeout =30
timeout_cnt =0

while timeout> timeout_cnt :
    
    #Recognitionプロジェクトの状態を取得
    response =rekognition_client.describe_project_versions(
        ProjectArn=PROJECT_ARN,
        VersionNames=[VERSION_NAME,]
    )
    
    #RecognitionプロジェクトのステータスJSONを読み取り
    for pro_state in response["ProjectVersionDescriptions"]:
        
        if pro_state["ProjectVersionArn"] == REKOGNITION_ARN :
            
            #停止済いおしくは停止中
            if pro_state["Status"] == "STOPPED" or pro_state["Status"] == "STOPPING" :
                print ("Is STOPPED now…")
                is_stop= True

            #起動中
            if pro_state["Status"] == "RUNNING":
                print ("Is RUNNING now…")
                is_running=True

            #起動開始中ためRUNNINGまで待機
            if pro_state["Status"] == "STARTING":
                print ("Is STARTING wait…")

            break
    
    #停止状態もしくは起動中を確認できたら無限ループから脱出
    if is_running or is_stop :
        
        if is_running :
            response = rekognition_client.stop_project_version(
                ProjectVersionArn=REKOGNITION_ARN,
            )
        print(response["Status"])
        is_stop =True
        break
    
    #10秒待機
    print("Sleep 10s …")
    sleep (10)
    timeout_cnt +=1

if not is_stop :
     print("STOP Error")
    